In [1]:
import os
os.chdir('../')

import warnings
warnings.filterwarnings('ignore')

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os

!nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"]="6"

Wed May 24 17:59:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    Off  | 00000000:1B:00.0 Off |                    0 |
| 30%   27C    P8    14W / 230W |  14334MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A5000    Off  | 00000000:1C:00.0 Off |                  Off |
| 30%   

### Hyperparams

In [4]:
n_mels = 8
n_outputs = 61
n_frames = 400
sr = 24000
fps = 30
train_csv_files = ['/Storage/speech/face/MySlate_5_박수철의_iPhone_30fps.csv']
train_wav_files = ['/Storage/speech/face/MySlate_5_박수철의_iPhone.wav']

test_csv_files = ['/Storage/speech/face/MySlate_6_박수철의_iPhone_30fps.csv']
test_wav_files = ['/Storage/speech/face/MySlate_6_박수철의_iPhone.wav']

### Model

In [5]:
from model.model_vqvae import Model
from utils.util import *
from tensorboardX import SummaryWriter

step = 0
device = 'cuda:0'

# Model
model = Model(in_dim=n_mels, out_dim=n_outputs, K=16, latent_dim=128)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
print('done')

done


### Load

In [6]:
save_dir = '/data/scpark/save/lips/train05.24-1/'
!mkdir -p $save_dir
!ls -lt $save_dir

writer = SummaryWriter(save_dir)

if False:
    step, model, _, optimizer = load(save_dir, 1000, model, None, optimizer)

total 0


In [7]:
import torch
from data.dataset import LipsDataset, Collate

dataset = LipsDataset(train_wav_files, train_csv_files, n_frames, n_mels=n_mels, sr=sr, fps=fps)
print('train dataset :', len(dataset))
train_loader = torch.utils.data.DataLoader(dataset, num_workers=1, shuffle=True, batch_size=8, 
                                           collate_fn=Collate(n_frames, n_mels))
print(train_loader)

dataset = LipsDataset(test_wav_files, test_csv_files, n_frames, n_mels=n_mels, sr=sr, fps=fps)
print('test dataset :', len(dataset))
test_loader = torch.utils.data.DataLoader(dataset, num_workers=1, shuffle=True, batch_size=8,
                                           collate_fn=Collate(n_frames, n_mels))
print(test_loader)

train dataset : 8816
test dataset : 1413


In [ ]:
from IPython import display
import librosa.display
import matplotlib.pyplot as plt

isnan = False
while True:
    if isnan:
        break
    for batch in train_loader:
        inputs = batch['inputs'].transpose(1, 2).to(device)
        targets = batch['outputs'].transpose(1, 2).to(device)
        
        model.train()
        model.zero_grad()
        outputs = model(inputs, targets)
        
        print(step)
        loss = 0
        for key in outputs.keys():
            if 'loss' in key:
                loss += outputs[key]
                print(key, outputs[key].item())
        if torch.isnan(loss):
            isnan = True
            break
        loss.backward()
        optimizer.step()
        print(step, loss.item())
        
        if step % 100 == 0:
            writer.add_scalar('train_loss', loss.item(), step)
        
        if step % 100 == 0:
            display.clear_output()
            
            losses = []
            for i, batch in enumerate(test_loader):
                if i >= 10:
                    break
                    
                inputs = batch['inputs'].transpose(1, 2).to(device)
                targets = batch['outputs'].transpose(1, 2).to(device)

                model.eval()
                with torch.no_grad():
                    outputs = model(inputs, targets)
                    
                loss = 0
                for key in outputs.keys():
                    if 'loss' in key:
                        loss += outputs[key]
                print('test :', i, loss.item())
                losses.append(loss)        
            
            test_loss = torch.stack(losses).mean().item()
            print('test_loss :', test_loss)
            writer.add_scalar('test_loss', test_loss, step)

        if step % 1000 == 0:
            save(save_dir, step, model, None, optimizer)
    
        step += 1

test : 0 2.9575023651123047
test : 1 2.9608356952667236
test : 2 2.959960460662842
test : 3 2.956312417984009
test : 4 2.957563638687134
test : 5 2.9602882862091064
test : 6 2.958806037902832
test : 7 2.957486629486084
test : 8 2.9605841636657715
test : 9 2.959831953048706
test_loss : 2.9589173793792725
saved /data/scpark/save/lips/train05.24-1/save_0
1
auto_encoding_loss 0.1386832743883133
commit_loss 0.0005454104393720627
zi_prediction_loss 2.758803129196167
1 2.898031711578369
2
auto_encoding_loss 0.126823291182518
commit_loss 0.00042621997999958694
zi_prediction_loss 2.757453203201294
2 2.884702682495117
3
auto_encoding_loss 0.12454825639724731
commit_loss 0.00038213879452086985
zi_prediction_loss 2.754619836807251
3 2.8795502185821533
4
auto_encoding_loss 0.1077672615647316
commit_loss 0.00041077076457440853
zi_prediction_loss 2.7521886825561523
4 2.8603668212890625
5
auto_encoding_loss 0.10594354569911957
commit_loss 0.000414617417845875
zi_prediction_loss 2.749389410018921
5 2.8

59 2.2755208015441895
60
auto_encoding_loss 0.03186721354722977
commit_loss 0.0023901250679045916
zi_prediction_loss 2.308554172515869
60 2.3428115844726562
61
auto_encoding_loss 0.02810298278927803
commit_loss 0.002637171186506748
zi_prediction_loss 2.265334129333496
61 2.296074390411377
62
auto_encoding_loss 0.026989353820681572
commit_loss 0.0024449059274047613
zi_prediction_loss 2.219562530517578
62 2.2489967346191406
63
auto_encoding_loss 0.028363564983010292
commit_loss 0.0025138165801763535
zi_prediction_loss 2.277529716491699
63 2.3084070682525635
64
auto_encoding_loss 0.02800101600587368
commit_loss 0.0025829430669546127
zi_prediction_loss 2.2564547061920166
64 2.287038564682007
65
auto_encoding_loss 0.03056773543357849
commit_loss 0.002606076654046774
zi_prediction_loss 2.1975960731506348
65 2.2307698726654053
66
auto_encoding_loss 0.02878398261964321
commit_loss 0.0027629893738776445
zi_prediction_loss 2.2333438396453857
66 2.2648909091949463
67
auto_encoding_loss 0.02927491